In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df, test_df]

In [4]:
print(train_df.columns.values)

In [5]:
train_df.head()

In [6]:
train_df.tail()

In [7]:
train_df.info()
test_df.info()

In [8]:
train_df.describe()

In [9]:
train_df.describe(include=['O'])

In [10]:
train_df[['Pclass', 'Survived']].groupby(by = 'Pclass', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [11]:
train_df[['Sex', 'Survived']].groupby(by = 'Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [12]:
train_df[['SibSp', 'Survived']].groupby(by = 'SibSp').mean().sort_values(by = 'Survived', ascending = False)

In [13]:
train_df[['Parch', 'Survived']].groupby(by = 'Parch').mean().sort_values(by = 'Survived', ascending = False)

In [14]:
graph = sns.FacetGrid(train_df, col = "Survived")
graph.map(sns.histplot, 'Age')

In [15]:
grid = sns.FacetGrid(train_df, col = "Survived", row = "Pclass")
grid.map(sns.histplot, "Age")
grid.legend

In [16]:
grid = sns.FacetGrid(train_df, row = 'Embarked', size = 4)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
grid.add_legend()

In [17]:
grid = sns.FacetGrid(train_df, row = 'Embarked', col = 'Survived', size = 4)
grid.map(sns.barplot, 'Sex', 'Fare')
grid.add_legend()

In [18]:
train_df = train_df.drop(['Ticket', 'Cabin'], axis = 1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis = 1)

In [19]:
combine = [train_df, test_df]
combine

In [20]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
pd.crosstab(index = train_df['Title'], columns = train_df['Sex'] )

In [21]:
#combine[0].head()
train_df.head()

In [22]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

train_df[['Title', 'Survived']].groupby(by='Title').mean()

In [23]:
title_mapping = {'Mr': 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

In [24]:
train_df = train_df.drop(['Name', 'PassengerId'], axis = 1)
test_df = test_df.drop(['Name'], axis = 1)

In [25]:
combine = [train_df, test_df]
train_df.shape, test_df.shape

In [26]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female': 1, 'male': 0}).astype(int)
    
train_df.head()

In [27]:
grid = sns.FacetGrid(train_df, row = 'Pclass', col = 'Sex')
grid.map(sns.histplot, 'Age')
grid.add_legend()

In [28]:
guess_ages = np.zeros((2,3))
guess_ages

In [29]:
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j + 1)]['Age'].dropna()
            
            age_guess = guess_df.median()
            
            guess_ages[i,j] = int(age_guess/0.5 + 0.5) * 0.5
    for i in range(0,2):
        for j in range(0,3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i,j]
            
    dataset['Age'] = dataset['Age'].astype(int)
    
train_df.head()

In [30]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(by = 'AgeBand').mean().sort_values(by = 'AgeBand', ascending = True)

In [31]:
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset["Age"] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset["Age"] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset["Age"] <= 64), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 64), 'Age'] = 4

test_df.tail(30)

In [32]:
train_df = train_df.drop(['AgeBand'], axis = 1)
combine = [train_df, test_df]
combine[0].head()

In [33]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(by = "FamilySize").mean().sort_values(by = 'Survived', ascending = False)

In [34]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
train_df[['IsAlone', 'Survived']].groupby(by = 'IsAlone').mean()

In [35]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis = 1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis = 1)
combine = [train_df, test_df]

In [36]:
train_df.head()

In [37]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head()

In [38]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [39]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

train_df[['Embarked','Survived']].groupby(by = 'Embarked').mean().sort_values(by = 'Survived', ascending = False)

In [40]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map({'S' : 0, "C": 1, "Q": 2}).astype(int)
    
train_df.head()

In [41]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace = True)
test_df.head()

In [42]:
test_df.info()

In [43]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand']).mean().sort_values(by = 'FareBand', ascending = True)

In [44]:
for dataset in combine:
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 31), 'Fare'] = 3
    
train_df = train_df.drop(['FareBand'], axis = 1)
combine = [train_df, test_df]

train_df.head(10)
test_df.head(10)

In [45]:
train_df.head(10)

In [47]:
X_train = train_df.drop("Survived", axis = 1)
Y_train = train_df["Survived"]
X_test = test_df.drop("PassengerId", axis =1).copy()
X_train.shape, Y_train.shape, X_test.shape

In [49]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

In [50]:
acc_log

In [51]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df['Correlation'] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by = 'Correlation', ascending = False)

In [52]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [54]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [55]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

In [56]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [57]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100 ,2)
acc_decision_tree

In [61]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [64]:
submission = pd.DataFrame({'PassengerID' : test_df["PassengerId"], "Survived": Y_pred})
submission.to_csv('C:/Users/forma/Desktop/Programming', index = False)